In [1]:
include("../LiPoSID.jl")

include("CtrlSID.jl")


using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


In [ ]:
#using Plots
#using Optim
#using Random

In [2]:
using DifferentialEquations

function kossak_obj(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

kossak_obj (generic function with 1 method)

In [17]:
function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end


function kossak_time_evolution(ρₒ, tₘₐₓ, dt, H0, C, Fᴼᴺᴮ)

    D(ρ) = sum(C .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
    
    # Define the differential equation
    function matrix_ode!(dρ, ρ, p, t)
        dρ .= -im * (H0 * ρ - ρ * H0) + D(ρ)
    end

    # Time span
    tspan = (0.0, tₘₐₓ)

    # Create an ODE problem
    prob = ODEProblem(matrix_ode!, ρₒ, tspan)

    # Solve the ODE using a suitable solver, e.g., Tsit5 for non-stiff problems
    sol = solve(prob, dt=dt, adaptive = false)

    # Access solution
    ρ_solution = sol.u
    t_solution = sol.t;
    
    return(t_solution, ρ_solution)

end 


kossak_time_evolution (generic function with 1 method)

### Kossakowski objective for two-level system


For the two level system we can use theorem originally proved by Kossakowski:

Kossakowski, Bull. Acad. Pol. Sci. Ser. Math. Astr. Phys. 21, 649 (1973).

and available also in classic GKS paper:

Completely positive dynamical semigroups of Nlevel systems
Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan
Citation: J. Math. Phys. 17, 821 (1976); doi: 10.1063/1.522979

Theorem 3.1. (two-level system)

A linear operator $L: M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$L = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where

(i) $H = \sum_1^3 h_i f_i, \quad h_i \in \R$

(ii) $f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$


(iii) $\{C_{ij}\}= \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} =$

$=\begin{pmatrix} 
-\gamma_1 + \gamma_2 + \gamma_3   & - i a_3   &  i a_2 \\
i a_3                 & \gamma_1 - \gamma_2 + \gamma_3 & -i a_1 \\
-i a_2                & i a_1      & \gamma_1 + \gamma_2 - \gamma_3 
\end{pmatrix}$

To write a Hamiltonian $H$ and than objective function we can use the set of orthonormal operators $f_j$

We use Pauli matrices $\sigma_x, \sigma_y, \sigma_z$ to choose 3 orthonormal $f_i$ for the two-level system to fulfill:

$f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$

In [4]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

3-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.5im; 0.0 - 0.5im 0.0 + 0.0im]
 [0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.5 + 0.0im]

In [5]:
@polyvar m_Re[1:3, 1:3] m_Im[1:3, 1:3]

M = m_Re + im * m_Im

3×3 Matrix{Polynomial{true, Complex{Int64}}}:
 m_Re₁₋₁ + (0+1im)m_Im₁₋₁  m_Re₁₋₂ + (0+1im)m_Im₁₋₂  m_Re₁₋₃ + (0+1im)m_Im₁₋₃
 m_Re₂₋₁ + (0+1im)m_Im₂₋₁  m_Re₂₋₂ + (0+1im)m_Im₂₋₂  m_Re₂₋₃ + (0+1im)m_Im₂₋₃
 m_Re₃₋₁ + (0+1im)m_Im₃₋₁  m_Re₃₋₂ + (0+1im)m_Im₃₋₂  m_Re₃₋₃ + (0+1im)m_Im₃₋₃

In [6]:
M * (M') == M * M'

true

In [7]:
Cˢʸᵐᵇ = M * (M')

3×3 Matrix{Polynomial{true, Complex{Int64}}}:
 m_Re₁₋₁² + m_Re₁₋₂² + m_Re₁₋₃² + m_Im₁₋₁² + m_Im₁₋₂² + m_Im₁₋₃²                                                                                                                                                                                      …  m_Re₁₋₁m_Re₃₋₁ + (0-1im)m_Re₁₋₁m_Im₃₋₁ + (0+1im)m_Re₃₋₁m_Im₁₋₁ + m_Re₁₋₂m_Re₃₋₂ + (0-1im)m_Re₁₋₂m_Im₃₋₂ + (0+1im)m_Re₃₋₂m_Im₁₋₂ + m_Re₁₋₃m_Re₃₋₃ + (0-1im)m_Re₁₋₃m_Im₃₋₃ + (0+1im)m_Re₃₋₃m_Im₁₋₃ + m_Im₁₋₁m_Im₃₋₁ + m_Im₁₋₂m_Im₃₋₂ + m_Im₁₋₃m_Im₃₋₃
 m_Re₁₋₁m_Re₂₋₁ + (0+1im)m_Re₁₋₁m_Im₂₋₁ + (0-1im)m_Re₂₋₁m_Im₁₋₁ + m_Re₁₋₂m_Re₂₋₂ + (0+1im)m_Re₁₋₂m_Im₂₋₂ + (0-1im)m_Re₂₋₂m_Im₁₋₂ + m_Re₁₋₃m_Re₂₋₃ + (0+1im)m_Re₁₋₃m_Im₂₋₃ + (0-1im)m_Re₂₋₃m_Im₁₋₃ + m_Im₁₋₁m_Im₂₋₁ + m_Im₁₋₂m_Im₂₋₂ + m_Im₁₋₃m_Im₂₋₃     m_Re₂₋₁m_Re₃₋₁ + (0-1im)m_Re₂₋₁m_Im₃₋₁ + (0+1im)m_Re₃₋₁m_Im₂₋₁ + m_Re₂₋₂m_Re₃₋₂ + (0-1im)m_Re₂₋₂m_Im₃₋₂ + (0+1im)m_Re₃₋₂m_Im₂₋₂ + m_Re₂₋₃m_Re₃₋₃ + (0-1im)m_Re₂₋₃m_Im₃₋₃ + (0+1im)m_Re₃₋₃m_Im₂₋₃ + m_Im₂₋₁m_Im₃₋₁ + 

In [8]:
@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

H0ˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

H0ᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(H0ᴷˢʸᵐᵇ) == 0
@assert H0ᴷˢʸᵐᵇ == H0ˢʸᵐᵇ

Hˢʸᵐᵇ = H0ˢʸᵐᵇ 

#@polyvar d_Re d_Im

#Vˢʸᵐᵇ =  [ 0             d_Re+im*d_Im
#           d_Re-im*d_Im  0.           ]    

#@polyvar γ[1:3]
#@polyvar a[1:3]



#Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]
#          im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
#          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

#κ₁ = -γ[1]+γ[2]+γ[3]
#κ₂ =  γ[1]-γ[2]+γ[3]
#κ₃ =  γ[1]+γ[2]-γ[3]

#constr1 = κ₁ + κ₂ + κ₃  
#constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - - a[1]^2 - a[2]^2 - a[3]^2
#constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2 


#constraints = [γ[1], γ[2], γ[3], constr1, constr2, constr3]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.5+0.0im)ϵ                       (0.5+0.0im)h_Re + (0.0+0.5im)h_Im
 (0.5+0.0im)h_Re + (0.0-0.5im)h_Im  (-0.5+0.0im)ϵ

In [9]:
constraints = []

Any[]

In [10]:

function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end


function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

In [11]:
function quadruple_time_points(time_points::Vector{Float64})::Vector{Float64}
    # Check if there are at least two points to interpolate between
    if length(time_points) < 2
        return time_points
    end

    new_time_points = Float64[]
    
    for i in 1:(length(time_points) - 1)
        # Current point
        t1 = time_points[i]
        # Next point
        t2 = time_points[i + 1]
        
        # Calculate the intermediate points
        delta = (t2 - t1) / 4
        t1_1 = t1 + delta
        t1_2 = t1 + 2 * delta
        t1_3 = t1 + 3 * delta
        
        # Add the points to the new list
        push!(new_time_points, t1)
        push!(new_time_points, t1_1)
        push!(new_time_points, t1_2)
        push!(new_time_points, t1_3)
    end
    
    # Add the last original point
    push!(new_time_points, time_points[end])
    
    return new_time_points
end

# Example usage
original_points = [1.0, 2.0, 3.0, 4.0]
new_points = quadruple_time_points(original_points)
println(new_points)

[1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0]


In [12]:
function LME_GEXY_timeevolution(evol_data_file_name, basis, Hˢⁱᵐ, γ)

    data(ρ) = [ρᵢ.data for ρᵢ in ρ]

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = convert.(Vector{Matrix{ComplexF64}}, ρᵍᵉˣʸ)
    tᵍ, tᵉ, tˣ, tʸ = quadruple_time_points.(convert.(Vector{Float64},tᵍᵉˣʸ)) #quadruple_time_points

    Jˢⁱᵐ = DenseOperator( basis, [0  sqrt(parse(Float64, γ))
                                  0  0. + 0im                  ] )

    Hˢⁱᵐ = DenseOperator(basis, Hermitian(Hˢⁱᵐ))
        
    tˢⁱᵐᵍ, ρˢⁱᵐᵍ  = timeevolution.master(tᵍ, DenseOperator(basis,Hermitian(ρᵍ[1])), Hˢⁱᵐ, [Jˢⁱᵐ])
    tˢⁱᵐᵉ, ρˢⁱᵐᵉ  = timeevolution.master(tᵉ, DenseOperator(basis,Hermitian(ρᵉ[1])), Hˢⁱᵐ, [Jˢⁱᵐ])
    tˢⁱᵐˣ, ρˢⁱᵐˣ  = timeevolution.master(tˣ, DenseOperator(basis,Hermitian(ρˣ[1])), Hˢⁱᵐ, [Jˢⁱᵐ])
    tˢⁱᵐʸ, ρˢⁱᵐʸ  = timeevolution.master(tʸ, DenseOperator(basis,Hermitian(ρʸ[1])), Hˢⁱᵐ, [Jˢⁱᵐ])

    ρˢⁱᵐᵍᵉˣʸ = ρˢⁱᵐᵍ, ρˢⁱᵐᵉ, ρˢⁱᵐˣ, ρˢⁱᵐʸ 
    tˢⁱᵐᵍᵉˣʸ = tˢⁱᵐᵍ, tˢⁱᵐᵉ, tˢⁱᵐˣ, tˢⁱᵐʸ

    return tˢⁱᵐᵍᵉˣʸ , data.(ρˢⁱᵐᵍᵉˣʸ)

end

LME_GEXY_timeevolution (generic function with 1 method)

In [13]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"
basis =  NLevelBasis(2)

Hˢⁱᵐ =  [ 25.133    0. + 0im
          0        -25.133   ] / 2

γ = "25.133"

LME_GEXY_timeevolution(evol_data_file_name, basis, Hˢⁱᵐ, γ);

In [14]:
function TrDist(ρ₁, ρ₂)
    A = ρ₁-ρ₂
    D = tr(sqrt(A'*A))/2
    if abs(imag(D))>1e-6
        throw(DomainError(D, "Trace distance is complex number"))
    else
        return(real(D))
    end
end 

TrDist (generic function with 1 method)

In [15]:
#tests_dir = "../TESTS/"
tests_dir = ""

ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

In [18]:
println(" SYSTEM IDENTIFICATION of simulated LME dt/4 w TSSOS and General KOSSAKOWSKI objective")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

tests_data_file_name = "POP_SIM_LME_TSSOS_GEN_C_KOS_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []

Hˢⁱᵐ =  [ 25.133    0. + 0im
          0        -25.133  ] / 2

for γᵢ in γ

    println("γ =  "*γᵢ)

    Jˢⁱᵐ = DenseOperator( basis, [0  sqrt(parse(Float64, γᵢ))
                                  0  0. + 0im                  ] )

    tᵍᵉˣʸ , ρᵍᵉˣʸ = LME_GEXY_timeevolution(evol_data_file_name, basis, Hˢⁱᵐ, γᵢ)
    
    polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

    @show minimum(abs.(coefficients(polyGEXY)))
    @show maximum(abs.(coefficients(polyGEXY)))

    #sol, opt_val, status = CtrlSID.cs_tssos(polyGEXY, constraints)
    #sol, opt_val, status = CtrlSID.jump(polyGEXY)
    sol, opt_val, status = CtrlSID.tssos(polyGEXY)

    print(" status:", status)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol); Cˢⁱᵈ = subs(Cˢʸᵐᵇ, sol)
    epsilon = subs(ϵ, sol)

    push!(Epsilons, epsilon)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   
        γ_group["epsilon"] = convert(Float64, epsilon)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)

    end

    println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        #Simulated LME 
        tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵐ), [Jˢⁱᵐ])
        #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

        ρₒ = ρₛ[1]
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        tˢⁱᵈ, ρˢⁱᵈ  = kossak_time_evolution(ρₒ, tᵉⁿᵈ, dt, Hˢⁱᵈ, Cˢⁱᵈ, fᴷᴼᴺᴮ)
        #bˢⁱᵈ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵈ])

        F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
        #Fˢᵖⁱⁿᵇᴼˢᴼⁿ= LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            #init_state_group["Fidelity_SB"] = convert.(Float64, Fˢᵖⁱⁿᵇᴼˢᴼⁿ)
            init_state_group["F"] = convert.(Float64, F)
            #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            #init_state_group["bloch_sid_lme"] = convert.(Float64, bˢⁱᵈ)
            #init_state_group["bloch_sim_lme"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
            init_state_group["time"] = tᵗˢᵗ
        end
        
        FminState = minimum(F)
        FmedianState = median(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Energy"] = convert.(Float64,Epsilons)
end


println(tests_data_file_name)

 SYSTEM IDENTIFICATION of simulated LME dt/4 w TSSOS and General KOSSAKOWSKI objective
γ =  0.079477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9301429115225457
γ =  0.25133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.8367085196494579
γ =  0.79477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.8268700153546078
γ =  2.5133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9999999999997238
γ =  7.9477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.99999999999991
γ =  25.133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9999999999938745
γ =  79.477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9999999999371022
γ =  251.33
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.9999999999817
POP_SIM_LME_TSSOS_GEN_C_KOS_2024-Jul-06_at_03-02.h5


 SYSTEM IDENTIFICATION of simulated LME dt/4 w TSSOS and General KOSSAKOWSKI objective
γ =  0.079477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.798871240629714
γ =  0.25133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.8072770344200042
γ =  0.79477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.8239545512321329
γ =  2.5133


LoadError: InterruptException:

 SYSTEM IDENTIFICATION of simulated LME w TSSOS and General KOSSAKOWSKI objective


γ =  0.079477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.92912994815291
γ =  0.25133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.8367083902116217
γ =  0.79477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.8268708114414818
γ =  2.5133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.999999999999724
γ =  7.9477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9999999999999448
γ =  25.133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9999999999992323
γ =  79.477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9999999999842069
γ =  251.33
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.9999999999993288
POP_SIM_LME_TSSOS_GEN_C_KOS_2024-Jul-05_at_15-55.h5


In [25]:
Epsilons

8-element Vector{Any}:
 25.12381650968752
 25.132891760649166
 25.133019332433236
 25.133000527885454
 25.132986539873293
 25.13287306748503
 25.13051886071538
 25.13172525512193

 SYSTEM IDENTIFICATION of simulated LME w JUMP and General KOSSAKOWSKI objective
γ =  0.079477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9317425753050486
γ =  0.25133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.8567073218410464
γ =  0.79477
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.8503122824847316
γ =  2.5133
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.8585038506930569
γ =  7.9477
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.8854280291214025
γ =  25.133
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9032195716090419
γ =  79.477
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.8767525201430664
γ =  251.33
 status:ITERATION_LIMIT
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity fo

In [19]:
# without constraints

 SYSTEM IDENTIFICATION of simulated LME w JUMP and General KOSSAKOWSKI objective
γ =  0.079477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9317425753050486
γ =  0.25133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.8567073218410464
γ =  0.79477
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.8503122824847316
γ =  2.5133
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.8585038506930569
γ =  7.9477
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.8854280291214025
γ =  25.133
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9032195716090419
γ =  79.477
 status:ALMOST_LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.8767525201430664
γ =  251.33
 status:ITERATION_LIMIT
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity fo

 SYSTEM IDENTIFICATION of simulated LME w CONSTRAINED JUMP and KOSSAKOWSKI objective
γ =  0.079477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9996420396228947
γ =  0.25133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9999999235652612
γ =  0.79477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9999996072317053
γ =  2.5133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9999991763041339
γ =  7.9477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9999996732440135
γ =  25.133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9999998423004073
γ =  79.477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9999991703220533
γ =  251.33
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.999999926645021
POP_

 SYSTEM IDENTIFICATION w CONSTRAINED JUMP and KOSSAKOWSKI objective
γ =  0.079477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9977046690410722
γ =  0.25133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9966807880531652
γ =  0.79477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9977004864049941
γ =  2.5133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9990498352408502
γ =  7.9477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9996748517732996
γ =  25.133
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9968645845457611
γ =  79.477
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9989415717391549
γ =  251.33
 status:LOCALLY_SOLVED
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.9993933098260761
POP_NEW_JUMP_KOS_202

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective
γ =  0.079477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9977046690410768
γ =  0.25133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9966807880531685
γ =  0.79477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9977004864050036
γ =  2.5133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.999049835240695
γ =  7.9477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9996748517732994
γ =  25.133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9968645845347491
γ =  79.477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9989415677938539
γ =  251.33
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.999393310891379
POP_NEW_TSSOS_KOS_2024-Jun-15_at_04-22.h5


In [18]:
println(tests_data_file_name)

POP_NEW_TSSOS_KOS_2024-Jun-15_at_04-22.h5
